In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from PIL import Image

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.models import resnet152
import pytorch_lightning as pl
import sys
from tqdm import tqdm
torch.cuda.init()
assert torch.cuda.is_initialized()
print(torch.cuda.get_device_properties(0))

In [ ]:
!{sys.executable} -m pip install slideflow[tf] cucim cupy-cuda11x
#!{sys.executable} export SF_SLIDE_BACKEND=libvips
!{sys.executable} -m pip install fastai
!{sys.executable} -m pip install nystrom_attention
!{sys.executable} -m pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0

In [11]:
slide_biop_df = pd.read_csv("/home/ngsci/datasets/brca-psj-path/contest-phase-2/slide-manifest-train.csv")
outcomes_df = pd.read_csv("/home/ngsci/datasets/brca-psj-path/contest-phase-2/csv-train/outcomes.csv")
slide_stage_df = slide_biop_df.merge(outcomes_df, on="biopsy_id")

def stage_to_int(stage):
    if stage == "0":
        return "0"
    elif stage == "IA" or stage == "IB":
        return "1"
    elif stage == "IIA" or stage == "IIB":
        return "2"
    elif stage == "IIIA" or stage == "IIIB" or stage == "IIIC":
        return "3"
    elif stage == "IV":
        return "4"
    else:
        return ""

slide_stage_df["stage"] = slide_stage_df["stage"].apply(stage_to_int)
slide_stage_df["dataset"]="train"
change=slide_stage_df.sample(frac = 0.3).index
slide_stage_df.loc[change,'dataset'] = "eval"

# subset columns, drop nans, reset index
labels_df = (
    slide_stage_df[["slide_id", "biopsy_id", "stage","dataset"]]
    .copy()
    .dropna(how="any")
    .reset_index(drop=True)
)

labels_df["stage"] = labels_df["stage"]
labels_df.rename(columns = {'slide_id':'patient'}, inplace = True)
labels_df.head(5)
labels_df.to_csv('annotations.csv',index=False)

In [ ]:
import slideflow as sf
from slideflow.model import build_feature_extractor
from slideflow.model import list_extractors
list_extractors()

In [ ]:
# Load a project and dataset.
'''
P = sf.Project(
    root='proj',
    annotations='annotations.csv',
    create=True
)
'''

'''
P.add_source(
  name="my_datasets",
  slides="/home/ngsci/datasets/brca-psj-path/ndpi/",
  tiles='/home/ngsci/datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/patches/'
)
'''

P = sf.load_project('proj')

In [ ]:
full_dataset = P.dataset(tile_px=256, tile_um='43x')
full_dataset.extract_tiles(save_tfrecords=True, enable_downsample=False, roi_method='ignore', skip_extracted=True)